# Вариант 2


*   Зависимая переменная - цена, предикторы - мощность, емкость бензобака, расход топлива в городе.
*   Задачи про гипотезы:
    - Чем больше мощность, тем больше цена.
    - Цена зависит от расхода топлива в городе.
    - Цена одновременно зависит от расхода топлива и емкости бензобака.
    
1. Построить модель линейной регрессии с указанными параметрами **своими средствами** (вместе со свободным коэффициентом), то есть пользоваться готовыми линейными моделями **нельзя** (максимум - для сравнения своего результата с готовой реализации, плюс можно использовать NumPy для матричных вычислений или специальные функции, решающие оптимизационные задачи)
2. Рассчитать оценки наименьших квадратов для параметров и остаточной дисперсии
3. Вычислить коэффициент детерминации
4. Построить доверительные интервалы для параметров модели и остаточной дисперсии
5. Проверить гипотезы при уровне значимости $\alpha = 0,05$ (формализовать основную и альтернативную гипотезу, рассчитать статистику критерия, вычислить критические значения, указать p-value)









# Загрузка датасета и необходимых модулей

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats

In [ ]:
!wget -O /content/cars.csv 'https://drive.google.com/uc?id=1vv2jGNp6EO8HHRoscDRQU90faR3j8iTN'

--2024-06-06 18:26:13--  https://drive.google.com/uc?id=1vv2jGNp6EO8HHRoscDRQU90faR3j8iTN
Resolving drive.google.com (drive.google.com)... 173.194.213.138, 173.194.213.101, 173.194.213.113, ...
Connecting to drive.google.com (drive.google.com)|173.194.213.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1vv2jGNp6EO8HHRoscDRQU90faR3j8iTN [following]
--2024-06-06 18:26:13--  https://drive.usercontent.google.com/download?id=1vv2jGNp6EO8HHRoscDRQU90faR3j8iTN
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.134.132, 2607:f8b0:400c:c00::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.134.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14505 (14K) [application/octet-stream]
Saving to: ‘/content/cars.csv’

/content/cars.csv   100%[===================>]  14.17K  --.-KB/s    in 0s      

2024-06-06 18:26:13 (92.5 M

In [ ]:
data = pd.read_csv('/content/cars.csv')
data.head()

,Manufacturer,Model,Type,Min.Price,Price,Max.Price,MPG.city,MPG.highway,AirBags,DriveTrain,...,Passengers,Length,Wheelbase,Width,Turn.circle,Rear.seat.room,Luggage.room,Weight,Origin,Make
0,Acura,Integra,Small,12.9,15.9,18.8,25,31,NaN,Front,...,5,177,102,68,37,26.5,11.0,2705,non-USA,Acura Integra
1,Acura,Legend,Midsize,29.2,33.9,38.7,18,25,Driver & Passenger,Front,...,5,195,115,71,38,30.0,15.0,3560,non-USA,Acura Legend
2,Audi,90,Compact,25.9,29.1,32.3,20,26,Driver only,Front,...,5,180,102,67,37,28.0,14.0,3375,non-USA,Audi 90
3,Audi,100,Midsize,30.8,37.7,44.6,19,26,Driver & Passenger,Front,...,6,193,106,70,37,31.0,17.0,3405,non-USA,Audi 100
4,BMW,535i,Midsize,23.7,30.0,36.2,22,30,Driver only,Rear,...,4,186,109,69,39,27.0,13.0,3640,non-USA,BMW 535i


In [ ]:
data.shape

(93, 27)

In [ ]:
data.columns

Index(['Manufacturer', 'Model', 'Type', 'Min.Price', 'Price', 'Max.Price',
       'MPG.city', 'MPG.highway', 'AirBags', 'DriveTrain', 'Cylinders',
       'EngineSize', 'Horsepower', 'RPM', 'Rev.per.mile', 'Man.trans.avail',
       'Fuel.tank.capacity', 'Passengers', 'Length', 'Wheelbase', 'Width',
       'Turn.circle', 'Rear.seat.room', 'Luggage.room', 'Weight', 'Origin',
       'Make'],
      dtype='object')

# Решение

Цель модели линейной регрессии – найти такие коэффициенты $\beta$, которые бы минимизировали разность квадратов между предсказанными и истинными значениями зависимой переменной.

$$Y=X\beta+E$$

## Построение модели линейной регрессии

Зависимая переменная - цена (`Price`), предикторы - мощность, емкость бензобака, расход топлива в городе (`Horsepower`, `Fuel.tank.capacity`, `MPG.city`)

Создадим матрицу признаков и вектор таргетов для этой задачи.

In [ ]:
x = data[['Horsepower', 'Fuel.tank.capacity', 'MPG.city']]
y = data['Price']

In [ ]:
x = np.c_[np.ones(x.shape[0]), x] # добавляем столбец единиц

In [ ]:
x

array([[  1. , 140. ,  13.2,  25. ],
       [  1. , 200. ,  18. ,  18. ],
       [  1. , 172. ,  16.9,  20. ],
       [  1. , 172. ,  21.1,  19. ],
       [  1. , 208. ,  21.1,  22. ],
       [  1. , 110. ,  16.4,  22. ],
       [  1. , 170. ,  18. ,  19. ],
       [  1. , 180. ,  23. ,  16. ],
       [  1. , 170. ,  18.8,  19. ],
       [  1. , 200. ,  18. ,  16. ],
       [  1. , 295. ,  20. ,  16. ],
       [  1. , 110. ,  15.2,  25. ],
       [  1. , 110. ,  15.6,  25. ],
       [  1. , 160. ,  15.5,  19. ],
       [  1. , 110. ,  16.5,  21. ],
       [  1. , 170. ,  20. ,  18. ],
       [  1. , 165. ,  27. ,  15. ],
       [  1. , 170. ,  23. ,  17. ],
       [  1. , 300. ,  20. ,  17. ],
       [  1. , 153. ,  18. ,  20. ],
       [  1. , 141. ,  16. ,  23. ],
       [  1. , 147. ,  16. ,  20. ],
       [  1. ,  92. ,  13.2,  29. ],
       [  1. ,  93. ,  14. ,  23. ],
       [  1. , 100. ,  16. ,  22. ],
       [  1. , 142. ,  20. ,  17. ],
       [  1. , 100. ,  16. ,  21. ],
 

In [ ]:
y

0     15.9
1     33.9
2     29.1
3     37.7
4     30.0
      ... 
88    19.7
89    20.0
90    23.3
91    22.7
92    26.7
Name: Price, Length: 93, dtype: float64

In [ ]:
x_transposed = np.transpose(x) #транспонированная матрица x

In [ ]:
x_transposed

array([[  1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,   1. ,
          1. ,   1. ,   1. ],
       [140. , 200. , 172. , 172. , 208. , 110. , 170. , 180. , 170. ,
        200. , 295. , 110. , 110. , 160. , 110. , 170. , 165. , 170. ,
        300. , 153. , 141. , 147. ,  92. ,  93. , 100. , 142. , 100. ,
        300. ,  92. , 214. ,  63. , 127. ,  96.

In [ ]:
x_multiplied = x_transposed @ x #умножаем транспонированную матрицу x на матрицу x

In [ ]:
x_multiplied

array([[9.300000e+01, 1.337600e+04, 1.549800e+03, 2.080000e+03],
       [1.337600e+04, 2.176206e+06, 2.341519e+05, 2.809480e+05],
       [1.549800e+03, 2.341519e+05, 2.681606e+04, 3.328350e+04],
       [2.080000e+03, 2.809480e+05, 3.328350e+04, 4.942600e+04]])

In [ ]:
x_inversed = np.linalg.inv(x_multiplied) # находим обратную матрицу

In [ ]:
x_inversed

array([[ 2.44661074e+00, -6.11081545e-04, -7.66262201e-02,
        -4.78873347e-02],
       [-6.11081545e-04,  8.47859514e-06, -6.58834982e-05,
         2.18880484e-05],
       [-7.66262201e-02, -6.58834982e-05,  3.49522337e-03,
         1.24548027e-03],
       [-4.78873347e-02,  2.18880484e-05,  1.24548027e-03,
         1.07235686e-03]])

In [ ]:
matrix = x_inversed @ x_transposed

In [ ]:
beta = matrix @ y

In [ ]:
beta

array([ 0.94881401,  0.12711435,  0.19477227, -0.13268236])

Посчитаем предсказанные значения
$\hat{y} = X\hat{\beta}$

In [ ]:
beta_transposed = np.transpose(np.matrix(beta))

In [ ]:
y_hat = x @ beta

In [ ]:
y_hat

array([17.99875793, 27.4893023 , 23.4504863 , 24.40121221, 28.5792817 ,
       15.20664581, 23.54318946, 26.1862414 , 23.69900728, 27.75466701,
       40.22007472, 14.574872  , 14.65278091, 21.78511529, 15.35880539,
       24.06541637, 25.19129761, 24.78241555, 40.72296411, 21.24956317,
       18.93659936, 20.09733253, 11.36653974, 12.44556606, 13.85759341,
       20.63889695, 13.99027576, 40.5513273 , 11.36653974, 29.00353847,
        6.79158762, 16.61163611, 13.32965878, 14.37630179, 15.40155779,
       21.48037699, 19.07484973, 26.60770335,  3.90130082, 10.82381086,
       21.19961241, 10.65960862, 18.87157493,  9.71507794, 16.46036156,
       11.51138286, 17.91588663, 38.41337911, 25.6799732 , 31.17356893,
       22.6153196 , 29.14999033, 10.09539625, 13.15068389, 21.36479618,
       22.08079222, 35.0028184 , 17.64423019, 29.61495001, 12.77052691,
       19.72975899, 11.36653974, 27.93830327, 13.65459802, 19.92846892,
       21.7829261 , 22.10406739, 20.42770007, 15.09344068, 24.06

## Оценка остаточной дисперсии

Рассчитаем оценку остаточной дисперсии по формуле
$$\hat{\sigma}^2 = \frac{e^Te}{n - k},$$
где $e = y - \hat{y}$, $n$ - количество наблюдений, $k$ - количество параметров модели.

In [ ]:
e = np.array(y) - np.array(y_hat)

In [ ]:
e

array([ -2.09875793,   6.4106977 ,   5.6495137 ,  13.29878779,
         1.4207183 ,   0.49335419,  -2.74318946,  -2.4862414 ,
         2.60099272,   6.94533299,  -0.12007472,  -1.174872  ,
        -3.25278091,  -6.68511529,   0.54119461,  -7.76541637,
        -8.59129761,  -5.98241555,  -2.72296411,  -2.84956317,
        -3.13659936,   9.40266747,  -2.16653974,  -1.14556606,
        -0.55759341,  -1.63889695,   1.60972424, -14.7513273 ,
         0.83346026,  -9.70353847,   0.60841238,  -6.51163611,
        -2.02965878,   1.52369821,  -1.40155779,  -1.58037699,
         1.12515027,  -5.70770335,   4.49869918,   1.67618914,
        -1.39961241,   1.44039138,  -1.37157493,  -1.71507794,
        -6.46036156,  -1.51138286,  -4.01588663,   9.48662089,
         2.3200268 ,   4.02643107,  11.6846804 ,   6.95000967,
        -1.79539625,  -1.55068389,  -4.86479618,  -2.98079222,
        -2.5028184 ,  14.25576981,  32.28504999,   1.32947309,
        -4.82975899,  -1.06653974,  -1.83830327,  -1.85

In [ ]:
e_transposed = np.transpose(e)

In [ ]:
e_transposed

array([ -2.09875793,   6.4106977 ,   5.6495137 ,  13.29878779,
         1.4207183 ,   0.49335419,  -2.74318946,  -2.4862414 ,
         2.60099272,   6.94533299,  -0.12007472,  -1.174872  ,
        -3.25278091,  -6.68511529,   0.54119461,  -7.76541637,
        -8.59129761,  -5.98241555,  -2.72296411,  -2.84956317,
        -3.13659936,   9.40266747,  -2.16653974,  -1.14556606,
        -0.55759341,  -1.63889695,   1.60972424, -14.7513273 ,
         0.83346026,  -9.70353847,   0.60841238,  -6.51163611,
        -2.02965878,   1.52369821,  -1.40155779,  -1.58037699,
         1.12515027,  -5.70770335,   4.49869918,   1.67618914,
        -1.39961241,   1.44039138,  -1.37157493,  -1.71507794,
        -6.46036156,  -1.51138286,  -4.01588663,   9.48662089,
         2.3200268 ,   4.02643107,  11.6846804 ,   6.95000967,
        -1.79539625,  -1.55068389,  -4.86479618,  -2.98079222,
        -2.5028184 ,  14.25576981,  32.28504999,   1.32947309,
        -4.82975899,  -1.06653974,  -1.83830327,  -1.85

In [ ]:
n = len(y)
k = x.shape[1]

In [ ]:
residuals = (e_transposed @ e) / (n - k)

In [ ]:
residuals

35.674752701766295

## Коэффициент детерминации

Найдем коэффициент детерминации по формуле $$R^2 = 1 - \frac{\sum{e^2}} {\sum{(y - \bar{y})^2}}$$

In [ ]:
r_squared = 1 - (np.sum(e ** 2)) / (np.sum((y - np.mean(y)) ** 2))

In [ ]:
(np.sum(e ** 2))

3175.052990457201

In [ ]:
(np.sum((y - np.mean(y)) ** 2))

8584.02129032258

In [ ]:
r_squared

0.630120559691915

## Доверительные интервалы для параметров

Построим доверительные интервалы для параметров модели

In [ ]:
se_beta = np.sqrt(np.diag(residuals * np.linalg.inv(x.T @ x)))
alpha = 0.05
t_crit = scipy.stats.t.ppf(1 - alpha/2, df=n - k)

conf_intervals = np.array([
    beta - t_crit * se_beta,
    beta + t_crit * se_beta
]).T

In [ ]:
conf_intervals

array([[-17.61452671,  19.51215472],
       [  0.09255738,   0.16167132],
       [ -0.5068626 ,   0.89640714],
       [ -0.52131866,   0.25595394]])

Построим доверительный интервал для достаточной дисперсии

In [ ]:
chi2_lower = scipy.stats.chi2.ppf(alpha/2, df=n - k)
chi2_upper = scipy.stats.chi2.ppf(1 - alpha/2, df=n - k)
conf_interval_residuals = [
    (n - k) * residuals / chi2_upper,
    (n - k) * residuals / chi2_lower
]

In [ ]:
conf_interval_residuals

[27.139738194746734, 49.002751480712206]

##Проверка гипотез

### Чем больше мощность, тем меньше цена.

$H_0$: Коэффициент $c_k$ = 0 (коэффициент при мощности равен нулю).

$H_1$: Коэффициент $c_k$ < 0 (коэффициент при мощности меньше нуля).

In [ ]:
t_stat_beta1 = beta[1] / se_beta[1]
p_value_beta1 = scipy.stats.t.cdf(t_stat_beta1, df=n - k)

In [ ]:
t_stat_beta1

7.308902076822737

In [ ]:
p_value_beta1

0.9999999999445062

Гипотеза не опровергнута.

### Цена зависит от расхода топлива в городе.

$H_0$: $c_k = 0$ (коэффициент при расходе топлива в городе равен нулю).

$H_1$: $c_k \ne 0$ (коэффициент при расходе топлива в городе не равен нулю).

In [ ]:
t_stat_beta3 = beta[3] / se_beta[3]
p_value_beta3 = 2 * (1 - scipy.stats.t.cdf(np.abs(t_stat_beta3), df=n - k))

In [ ]:
t_stat_beta3

-0.6783643693727525

In [ ]:
p_value_beta3

0.49930073341163084

Гипотеза не опровергнута.

###Цена одновременно зависит от расхода топлива и емкости бензобака.

$H_0$: Все коэффициенты модели равны 0.

$H_1$: Один или более из коэффициентов модели не равен 0.

In [ ]:
f_stat = ((np.sum((y - np.mean(y)) ** 2) - np.sum(residuals ** 2)) / 2) / (np.sum(residuals ** 2) / (n - k))
p_value_f = 1 - scipy.stats.f.cdf(f_stat, dfn=2, dfd=n - k)

In [ ]:
p_value_f

1.1102230246251565e-16

Гипотеза опровергнута.